# <span style="color:red"> Análisis Numérico I - FIUBA (Cátedra Tarela)</span>
# <span style="color:red">Ecuaciones No Lineales</span>

## <span style="color:red">1er cuatrimestre 2020</span>

In [1]:
import numpy as np
import sympy as sp
import math

import matplotlib.pyplot as plt
from sympy.calculus.util import continuous_domain
from sympy.solvers import solve
from scipy.linalg import solve as solve_matrix
from scipy.integrate import quad
from scipy.linalg import lu

# 0. Funciones auxiliares

In [2]:
def print_matriz(matriz, spacing=15, rounding=-1):
    
    space_str = "{: >" + str(spacing)+ "}" + " "*3 + "\t"
    line = space_str * len(matriz)
    
    for fila in matriz:
        
        fila_modificada = fila
        if (rounding!=-1):
            fila_modificada = [round(x, rounding) for x in fila_modificada]  
            
        fila_string = [str(x) for x in fila_modificada]        
        print('|'+ line.format(*fila_string) + "|")

In [3]:
def print_vector(vector, spacing=4, rounding=-1):
    
    space_str = "{: >" + str(spacing)+ "}" + " "*3
    line = space_str * len(vector)
    
    vector_modificado = vector
    if (rounding!=-1):
            vector_modificado = [round(x, rounding) for x in vector_modificado]  
    
    vector_string = [str(x) for x in vector_modificado]    
    print('|'+ line.format(*vector_string) + "|")

In [4]:
def resolver_sistema_ecuaciones(A,b):
    A_bis = np.array(A, dtype = 'float')
    b_bis = np.array(b, dtype = 'float')
    return list(solve_matrix(A_bis,b_bis))

In [5]:
def matriz_aumentada(A,b):
    
    m = np.array(A +[b])

    A_bis, b_bis = lu(m, permute_l=True)
    
    return A_bis, b_bis

In [6]:
def permutar_filas(A,b,pos1,pos2):
    
    fil_aux = A[pos1]
    A[pos1] = A[pos2]
    A[pos2] = fil_aux
    
    b_aux = b[pos1]
    b[pos1] = b[pos2]
    b[pos2] = b_aux
    
    return A,b

In [7]:
def permutar_columnas(A,x,pos1,pos2):
    
    col_aux = []
    for fil in A:
        col_aux.append(fil[pos1])
    
    n = 0
    for fil in range(len(A)):
        A[fil][pos1] = A[fil][pos2]
        A[fil][pos2] = col_aux[n]
        n +=1
        
    x_aux = x[pos1]
    x[pos1] = x[pos2]
    x[pos2] = x_aux

    return A,x

In [8]:
def indice_fila_mayor(A,n):
    
    
    fil_max = 0
    
    for fil in range(1, len(A)):
        if(A[fil][n] > A[fil_max][n]):
            fil_max = fil
    
    return fil_max

In [9]:
def indice_valor_mayor(A):
    
    fil_max = 0
    col_max = 0
    
    for fil in range(0, len(A)):
        for col in range(0,len(A[0])): 
            if(A[fil][col] > A[fil_max][col_max]):
                fil_max = fil
                col_max = col
    
    return fil_max, col_max

In [10]:
def resolver_matriz_triangulada(A,b):
    
    cant_fil = len(A)
    cant_col = len(A[0])
    
    res = [0]*cant_col
    
    for i in range(cant_fil):
        suma = sum([res[j]* A[cant_fil-i-1][j] for j in range(cant_col)])
        res[-i-1] = (b[-i-1] - suma) / A[cant_fil-i-1][cant_col-i-1]
    
    return res

In [11]:
def get_x(A):
    return ["x{}".format(i+1) for i in range(len(A[0]))]

# 1. MÉTODOS DIRECTOS

## 1.1 Eliminación de Gauss

### 1.1.1 Calculo

#### SIN PIVOTEO

In [12]:
#Pre condición: sistema linialmente dependiente

def triangular_sin_pivoteo(A,b,x=None):
    
    if(x==None):x = get_x(A)
    
    cant_fil = len(A)
    cant_col = len(A[0])
    
    for fil in range(cant_fil-1):
        
        for sub_fil in range(fil+1, cant_col):
            
            n = 1
            while(A[fil][fil]==0):
                A, b = permutar_filas(A,b,fil,fil + n)
                n += 1
                if(fil+n == cant_fil): break
            if(fil+n == cant_fil):break
                
            pivote = A[sub_fil][fil]/A[fil][fil]
            
            A[sub_fil] = A[sub_fil][:fil]+[A[sub_fil][p] - pivote*A[fil][p] for p in range(fil,cant_col)]
            b[sub_fil] = b[sub_fil] - pivote*b[fil]
            
    return A,b,x

#### PIVOTEO PARCIAL

In [13]:
def _permutar_fila_mayor(A,b,n):
    
    fil_max = indice_fila_mayor(A[n:],n) + n
    A,b = permutar_filas(A,b,n,fil_max)
    
    return A,b

In [14]:
#Pre condición: sistema linialmente dependiente

def triangular_pivoteo_parcial(A,b,x=None):
    
    if(x==None):x = get_x(A)
    
    cant_fil = len(A)
    cant_col = len(A[0])
    
    for fil in range(cant_fil-1):
                
        A,b = _permutar_fila_mayor(A,b,fil)

        for sub_fil in range(fil+1, cant_col):
                
            pivote = A[sub_fil][fil]/A[fil][fil]
            
            A[sub_fil] = A[sub_fil][:fil]+[A[sub_fil][p] - pivote*A[fil][p] for p in range(fil,cant_col)]
            b[sub_fil] = b[sub_fil] - pivote * b[fil]
    
    return A,b,x

#### PIVOTEO TOTAL O COMPLETO

In [15]:
def _indice_valor_mayor(A,n):
    B = A[n:]
    B = [fil[n:] for fil in B]
    
    fil_max, col_max = indice_valor_mayor(B)
    
    return fil_max+n, col_max+n

In [16]:
def _permutar_valor_mayor(A,b,x,n):
    
    fil_max, col_max = _indice_valor_mayor(A,n)
    
    A,b = permutar_filas(A,b,n,fil_max)
    
    A,x = permutar_columnas(A,x,n,col_max)
        
    return A,b,x

In [17]:
#Pre condición: sistema linialmente dependiente

def triangular_pivoteo_total(A,b,x=None):
    
    if(x==None):x = get_x(A)
    
    cant_fil = len(A)
    cant_col = len(A[0])
    
    for fil in range(cant_fil-1):
                
        A,b,x = _permutar_valor_mayor(A,b,x,fil)
    
        for sub_fil in range(fil+1, cant_col):
                
            pivote = A[sub_fil][fil]/A[fil][fil]
            
            A[sub_fil] = A[sub_fil][:fil]+[A[sub_fil][p] - pivote*A[fil][p] for p in range(fil,cant_col)]
            b[sub_fil] = b[sub_fil] - pivote*b[fil]
    
    return A,b,x

### 1.1.2 Errores

In [18]:
def cota_error_x(A,dA,db, x):
    inversa = np.linalg.inv(A)
    A = np.array(A)
    dA = np.array(dA)
    db = np.array(db)
    x = np.array(x)
    return abs(inversa.dot(db) - inversa.dot(dA.dot(x)))

In [19]:
def numero_de_condicion_K(A):
    norma_inf_inversa_A = np.linalg.norm(np.linalg.inv(A), ord = np.inf)
    norma_inf_A = np.linalg.norm(A, ord = np.inf)
    
    return norma_inf_A * norma_inf_inversa_A

In [20]:
def cota_error_relativo_norma_x(A,dA,b,db):
    norma_b = np.linalg.norm(b, ord = np.inf)
    norma_db = np.linalg.norm(db, ord = np.inf)
    norma_A = np.linalg.norm(A, ord = np.inf)
    norma_dA = np.linalg.norm(dA, ord = np.inf)
    
    return numero_de_condicion_K(A)*(norma_db/norma_b+norma_dA/norma_A)

In [21]:
def cota_error_absoluto_norma_x(A,x,unidad_de_maquina):
    
    norma_x = np.linalg.norm(x, ord = np.inf)
    
    return numero_de_condicion_K(A)*norma_x*2*unidad_de_maquina

In [22]:
def cota_error_de_redondeo(A,b,x_monio):
    K = numero_de_condicion_K(A)
    norma_b = np.linalg.norm(b, ord = np.inf)
    norma_R = np.linalg.norm(residuo(b, x_monio, A), ord = np.inf)
    return K*(norma_R/norma_b)

In [23]:
def residuo(b, x_monio, A):
    
    b_ = np.array(b).astype('float64') 
    
    x_monio = np.array(x_monio)
    A = np.array(A)
    b_monio = A.dot(np.array(x_monio)).astype("float32")

    return [abs(b_[i] - b_monio[i]) for i in range(len(b))]

### 1.1.3 Ejemplos

##### Gauss sin pivoteo

In [24]:
A = [[1,2,3],[4,5,6],[2,8,11]]
dA = [[0.1,0.1,0.02],[0.05,0.06,0.3],[0.1,0.3,0.01]]
b = [1,3,5]
db = [0.001,0.002,0.02]
      
A_bis, b_bis, x = triangular_pivoteo_parcial(A,b)
x_monio = resolver_matriz_triangulada(A_bis, b_bis)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

x1 = -0.22222222222222232
x2 = 1.4444444444444442
x3 = -0.5555555555555554


In [25]:
numero_de_condicion_K(A)

56.06060606060604

In [26]:
cota_error_x(A,dA,db, x_monio)

array([0.34856594, 1.38779349, 0.94246914])

##### Gauss con pivoteo parcial

In [27]:
A = [[1,2,3],[4,5,6],[2,8,11]]
dA = [[0.1,0.1,0.02],[0.05,0.06,0.3],[0.1,0.3,0.01]]
b = [1,5,6]
db = [0.001,0.002,0.02]
      
A_bis, b_bis, x = triangular_sin_pivoteo(A,b)
x_monio = resolver_matriz_triangulada(A_bis, b_bis)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

x1 = -0.11111111111111072
x2 = 3.222222222222222
x3 = -1.7777777777777777


In [28]:
cota_error_x(A,dA,db, x_monio)

array([0.26307407, 0.7277037 , 0.30592593])

In [29]:
cota_error_relativo_norma_x(A,dA,b,db)

0.8875

In [30]:
residuo(b, x_monio, A)

[0.0, 0.0, 1.5894571969710114e-07]

In [31]:
cota_error_de_redondeo(A,b,x_monio)

5.364418039777162e-07

##### Gauss con pivoteo total

In [32]:
A = [[1,2,3],[4,5,6],[2,8,11]]
b = [1,5,6]

A_bis,b_bis,x = triangular_pivoteo_total(A,b)
x_monio = resolver_matriz_triangulada(A_bis, b_bis)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

x3 = -1.7777777777777783
x1 = -0.11111111111111167
x2 = 3.222222222222223
